<a href="https://www.kaggle.com/code/momohmed2006/olympic-sports-data-preprocessing-1896-2014?scriptVersionId=279101870" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Project - Olympic Sports Analysis [Data Pre-processing]

- You can find the full project & the dataset at: https://www.kaggle.com/the-guardian/olympic-games
- In this project, we will consider these topics:
    - Data Cleaning & Manipulation
    - Data Grouping & Aggregation
    - Data Reshaping & Pivoting
    - Data Merging, Joining, & Concatenation

## Olympic Sports and Medals, 1896-2014
Which countries and athletes have won the most medals at the Olympic games?

### Importing libraries & data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
sns.set()

In [ ]:
summer = pd.read_csv('summer.csv')

In [ ]:
winter = pd.read_csv('winter.csv')

In [ ]:
countries = pd.read_csv('dictionary.csv')

### Inspecting Datasets

In [ ]:
summer.head()

In [ ]:
summer.info()

In [ ]:
winter.head()

In [ ]:
winter.info()

In [ ]:
countries.head()

In [ ]:
countries.info()

In [ ]:
# Listing all the missing data in the 'Countries' DataFrame
countries[countries.isna().any(axis= 1)].reset_index(drop= True)

---

### Proposed Questions

- ***Analysing all Summer editions data***
    - Can you find the __highest__ male / female __athletes__ of all time in the Summer editions?
    - Find the highest __athletes__ regarding to each __medal type__ in the Summer editions?

- ***Which are the most successful countries in both Summer and Winter editions?***
    - What are the __Top 10__ Countries by __total medals__?
    - __Split__ the total medals of Top 10 Countries into __Summer / Winter__. Are there typical Summer/Winter Games Countries? 
    - __Split__ the total medals of Top 10 Countries into __Gold, Silver, Bronze__. 


---

- ***Analysing all Summer editions data***
    - Can you find the __highest__ male / female __athletes__ of all time in the Summer editions?
    - Find the highest __athletes__ regarding to each __medal type__ in the Summer editions?

In [ ]:
# Modify the full name of the athelets in witer & summer editions 
def modify_name(name):
    split_name = name.split(",")
    if len(split_name) == 2:
        return split_name[1].strip() + " " + split_name[0].strip()
    else:
        return name
summer["Athlete"] = summer["Athlete"].apply(modify_name).str.title()
winter["Athlete"] = winter["Athlete"].apply(modify_name).str.title()

In [ ]:
summer[["Athlete"]].head()

In [ ]:
winter[["Athlete"]].head()

##### Q. Can you find the highest male / female athletes of all time in the Summer editions

In [ ]:
summer_medal_counts = summer.groupby(['Athlete', 'Gender'])["Medal"].size().sort_values(ascending= False).reset_index(name= "Medals_count")
summer_medal_counts.head()

In [ ]:
# Adding the countries column to our DataFrame
merged_country_summer = summer.merge(countries, left_on="Country", right_on="Code", how="left")
merged_country_summer.head()

In [ ]:
merged_country_summer = merged_country_summer[["Year", "City", "Sport", "Discipline", "Athlete",
                                               "Code", "Gender", "Event", "Medal", "Country_y"]]
merged_country_summer = merged_country_summer.rename(columns={"Country_y": "Country"})
merged_country_summer.head()

In [ ]:
# The highest male of all summer editions 
summer_medal_counts.loc[0, "Athlete"]

In [ ]:
## His total number of Medals
# The highest male of all summer editions 
summer_medal_counts.loc[0, "Medals_count"]

In [ ]:
# The highest female of all summer editions 
summer_medal_counts.loc[1, "Athlete"]

In [ ]:
summer_medal_counts.loc[1, "Medals_count"]

### Q. Find the highest athletes regarding to each medal type in the Summer editions

In [ ]:
summer_medal_type_counts = summer.groupby(['Athlete', 'Gender',"Medal"])["Medal"].size().sort_values(ascending= False).reset_index(name= "Medals_count")
medal_order = {"Gold":0, "Silver":1, "Bronze":2}
summer_medal_type_counts["Medal"] = summer_medal_type_counts["Medal"].map(medal_order)
summer_medal_type_counts_top = summer_medal_type_counts.sort_values(by=["Medal", "Medals_count"], ascending=[True, False]).groupby("Medal").head(1)
summer_medal_type_counts_top["Medal"] = summer_medal_type_counts_top["Medal"].replace({0:"Gold", 1:"Silver", 2:"Bronze"})
summer_medal_type_counts_top

In [ ]:
# Let's discover what dow 'Michael phelps' have of medals
merged_country_summer[merged_country_summer["Athlete"] == "Michael Phelps"]

In [ ]:
men_athlete = summer[summer["Gender"] == "Men"]

In [ ]:
summer_sport_type_counts = men_athlete.groupby(['Athlete', 'Medal'])["Sport"].size().sort_values(ascending= False).reset_index()
summer_sport_type_counts

In [ ]:
summer_medal_counts = summer.groupby(["Athlete", "Medal"])["Sport"].size().reset_index(name="Medals_count")
summer_medal_counts = summer_medal_counts.sort_values(by=["Medals_count"], ascending=False)
top_per_medal = summer_medal_counts.groupby("Medal").head(1)
top_per_medal

### Q. Calculate the medals per each country for the best male and females athletes in all of the Summer editions and visualize the results

In [ ]:
medals_country = merged_country_summer.pivot_table(index= "Country", columns= "Medal", values= "Athlete" , aggfunc= "count").fillna(0.0)
medals_country

In [ ]:
top_medals_country = medals_country.sort_values(by=["Gold", "Silver", "Bronze"], ascending=[False, False, False])
top_medals_country.head(11)

---

#### Q. Which are the most successful countries in both Summer and Winter editions?
- What are the __Top 10__ Countries by __total medals__?
- __Split__ the total medals of Top 10 Countries into __Summer / Winter__. Are there typical Summer/Winter Games Countries? 
- __Split__ the total medals of Top 10 Countries into __Gold, Silver, Bronze__. 

In [ ]:
merged_country_summer.head()

In [ ]:
winter.head()

In [ ]:
countries.head()

In [ ]:
s_w_merge = pd.concat([merged_country_summer, winter], keys=["Summer", "Winter"], names=["Edition"]).reset_index(level="Edition")
s_w_merge

In [ ]:
s_w_country_merge = pd.merge(s_w_merge, countries, how="left", left_on="Country", right_on="Code")
s_w_country_merge = s_w_country_merge.iloc[:, 0:13]
s_w_country_merge

In [ ]:
s_w_country_merge = s_w_country_merge.drop(columns= ["Code_x","Code_y"])
s_w_country_merge

In [ ]:
s_w_country_merge = s_w_country_merge.rename(columns= {"Country_x":"Code", "Country_y":"Country"})
s_w_country_merge

#### Remove Rows from Olympics where the Country code is unknown 

In [ ]:
s_w_country_merge_code_nan = s_w_country_merge[s_w_country_merge["Code"].isna()]
s_w_country_merge_code_nan

In [ ]:
s_w_country_merge = s_w_country_merge[s_w_country_merge["Code"].notna()]
s_w_country_merge

In [ ]:
s_w_country_merge.info()

In [ ]:
s_w_country_merge

#### Convert the column Medal into an ordered Categorical column ("Bronze" < "Silver" < "Gold")

In [ ]:
s_w_country_merge["Medal"] = pd.Categorical(s_w_country_merge["Medal"], 
                                            categories= ["Bronze","Silver","Gold"],ordered= True)
s_w_country_merge = s_w_country_merge.sort_values("Medal")

In [ ]:
s_w_country_merge.info()

In [ ]:
s_w_country_merge["Medal"]

---

#### 3] Data Analysis & Visualization (EDA)

#### Q. What are the Top 10 Countries by total medals?

In [ ]:
s_w_country_merge

In [ ]:
top_countries_medal = s_w_country_merge.groupby("Country")["Medal"].count().sort_values(ascending= False)
top_countries_medal

In [ ]:
top_10_countries = top_countries_medal.head(10)
top_10_countries

In [ ]:
plt.figure(figsize= (15,8))
sns.barplot(data= top_10_countries)

plt.title("Top 10 Countries")
plt.xlabel("Country")
plt.ylabel("Total Medals")
plt.show()

In [ ]:
s_w_country_merge.reset_index(drop= True)

#### Q. Split the total medals of Top 10 Countries into Summer / Winter. Are there typical Summer/Winter Games Countries?

In [ ]:
top10_countries = (
    s_w_country_merge.groupby("Country")["Medal"].count()
    .sort_values(ascending=False)
    .head(10)
    .index
)

In [ ]:
top10_df = s_w_country_merge[s_w_country_merge["Country"].isin(top10_countries)]
top10_df

In [ ]:
top_10_countries = top_10_countries.reset_index()
top_10_countries

In [ ]:
plt.figure(figsize=(15,8))
sns.barplot(data= top_10_countries, x= "Country", y= "Medal", hue= "Country")

plt.title("Top 10 Countries")
plt.xlabel("Country")
plt.ylabel("Count")
plt.show()

In [ ]:
top10_count = top10_df.groupby(["Country", "Edition"])["Medal"].count().sort_values(ascending= False)
top10_count = top10_count.reset_index()

plt.figure(figsize=(15,8))
sns.barplot(data= top10_count, x= "Country", y= "Medal", hue= "Edition", legend= True)

plt.title("Top 10 Countries")
plt.xlabel("Country")
plt.ylabel("Count")
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
sns.barplot(data= top_10_countries, x= "Country", y= "Medal", hue= "Country", legend= True, width= 0.1)

plt.title("Top 10 Countries")
plt.xlabel("Country")
plt.ylabel("Count")
plt.show()

#### Q. Split the total medals of Top 10 Countries into Gold, Silver, Bronze

In [ ]:
country_order = top10_df.groupby("Country")["Medal"].count().sort_values(ascending=False).index

medal_palette = {'Gold': '#FFD700','Silver': '#C0C0C0','Bronze': '#CD7F32'}
sns.set_palette(sns.color_palette([medal_palette[m] for m in ['Gold', 'Silver', 'Bronze']]))

plt.figure(figsize=(15,8))
sns.countplot(data=top10_df, x="Country", hue="Medal", order=country_order, palette= medal_palette)

plt.title("Top 10 Countries")
plt.xlabel("Country")
plt.show()

In [ ]:
country_order = top10_df.groupby("Country")["Medal"].count().sort_values(ascending=False).index

plt.figure(figsize=(15,8))
sns.countplot(data= top10_df, x= "Medal", hue= "Country", legend= True, palette= "Set2")

plt.title("Top 10 Countries")
plt.xlabel("Country")
plt.ylabel("Count")
plt.show()

---

# THANK YOU!